In [87]:
import pickle
import easygraph as eg

def load_graph_from_pickle(file_path):
    """从 Pickle 文件加载 EasyGraph 图"""
    with open(file_path, "rb") as f:
        return pickle.load(f)

# 示例：从 Pickle 文件加载图
graph = load_graph_from_pickle("graph.pkl")


In [88]:
from pyvis.network import Network
import math
import json

def visualize_single_base_model(eg_graph, base_model, output_file="single_model_graph.html", view_type=1):
    """
    可视化指定 base_model 的子图，支持不同的视图类型。
    Args:
        eg_graph: EasyGraph 的 DiGraph 对象。
        base_model: 要可视化的根节点（base_model）。
        output_file: 输出 HTML 文件的路径。
        view_type: 可视化模式，1：完整网络，2：同作者折叠，3：过滤企业，4：按类别展示
    """
    # 初始化 PyVis 图
    net = Network(height="1000px", width="100%", notebook=True, directed=True)

    # 边的颜色对应衍生类型
    edge_colors = {
        "adapter": "blue",
        "finetune": "green",
        "merge": "red",
        "quantized": "purple",
    }

    # 添加右侧侧边栏（显示节点详细信息）
    sidebar_html = """
    <div id="details" style="position: absolute; right: 10px; top: 10px; width: 450px; height: 700px; overflow: auto; background: #f9f9f9; border: 1px solid #ddd; padding: 10px;">
        <h4>Model Details</h4>
        <pre id="node-details">Click a node to see details</pre>
    </div>
    <script type="text/javascript">
        // 监听节点点击事件
        network.on("selectNode", function (params) {
            var nodeId = params.nodes[0];  // 获取节点 ID
            var nodeDetails = nodes.get(nodeId);  // 获取节点数据
            document.getElementById("node-details").innerHTML = nodeDetails.things_to_show_on_sidebar;  // 更新侧边栏内容
        });
    </script>
    """

    # 定义一个辅助函数，用于添加节点到图中
    def add_node_to_net(node, node_attrs, eg_graph):
        influence = node_attrs.get("influence", 1.0)
        pic = node_attrs.get("pic", "https://huggingface.co/front/assets/huggingface_logo-noborder.svg")
        downloads = node_attrs.get("downloads", 0)
        likes = node_attrs.get("likes", 0)
        author = node_attrs.get("author", "unknown")
        author_type = node_attrs.get("author_type", "unknown")
        
        # 判断模型的category
        # 查找指向当前节点的前驱节点（即入边）
        predecessors = list(eg_graph.predecessors(node))  # 获取所有指向该节点的前驱节点
        if predecessors:
            # 如果有前驱节点，使用前驱节点到当前节点的边的type属性作为category
            edge_type = eg_graph[predecessors[0]][node].get("type", "unknown")  # 获取第一条边的type作为示例
            category = f"{edge_type} of {predecessors[0]}"  # 比如： "adapter of base-model" 或者 "finetune of model-x"
        else:
            # 如果没有前驱节点，默认为 base model
            category = "base model"
        
        # 构建 HuggingFace 模型的网址
        model_url = f"https://huggingface.co/{node}"
        author_url = f"https://huggingface.co/{author}"

        # 构建用于展示的HTML内容，用于存进结点当作属性
        node_info_html = f"""
        <div style="font-family: Arial, sans-serif;">
            <img src="{pic}" style="width: 50px; height: 50px; border-radius: 5px; float: left; margin-right: 15px;" />
            <h4>{node}</h4>
            <p><strong>Author:</strong><a href= "{author_url}" >{author}</a> ({author_type})</p>
            <p><strong>Category:</strong> {category}</p>
            <p><strong>Downloads:</strong> {downloads}</p>
            <p><strong>Likes:</strong> {likes}</p>
            <p><strong>Influence:</strong> {influence}</p>
            <p><strong>Model URL:</strong> <a href="{model_url}" target="_blank">模型地址</a></p>
        </div>
        """
        
        net.add_node(
            node,
            label=node,
            size=math.sqrt(influence),
            title=f"{node}",  # 添加HTML格式的节点信息
            shape="image",
            image=pic,
            things_to_show_on_sidebar=node_info_html  # 添加到侧边栏的数据html代码
        )


    # 深度优先遍历，从 base_model 开始构建子图
    visited = set()  # 用于避免重复访问节点
    stack = [base_model]  # 使用栈模拟深度优先遍历
    author_models = {}  # 存储每个作者的模型，用于折叠

    while stack:
        node = stack.pop()
        if node in visited:
            continue
        visited.add(node)

        node_attrs = eg_graph.nodes[node]
        author = node_attrs.get("author", "unknown")
        
        # 根据选择的视图类型来决定如何处理模型节点
        if view_type == 2:  # 折叠同一个作者的模型
            if author not in author_models:
                author_models[author] = []
            author_models[author].append(node)
            continue  # 不添加节点，等到折叠处理

        # 添加节点到 PyVis 图
        if node not in net.get_nodes():
            add_node_to_net(node, node_attrs, eg_graph)  # 传递 eg_graph 参数

        # 遍历所有子节点（出边）
        for successor in eg_graph.successors(node):
            if successor not in visited and successor not in stack:
                stack.append(successor)

            # 确保目标节点添加到图中
            if successor not in net.get_nodes():
                successor_attrs = eg_graph.nodes[successor]
                add_node_to_net(successor, successor_attrs, eg_graph)  # 传递 eg_graph 参数

            # 添加边
            edge_data = eg_graph[node][successor]
            edge_type = edge_data.get("type", "unknown")
            edge_color = edge_colors.get(edge_type, "black")
            net.add_edge(node, successor, color=edge_color, title=edge_type)

    # 处理视图类型 2: 同一作者的模型折叠
    if view_type == 2:
        for author, models in author_models.items():
            author_size = sum(eg_graph.nodes[m].get("influence", 1.0) for m in models) / len(models)
            author_node = f"{author}_group"
            net.add_node(
                author_node,
                label=author,
                size=math.sqrt(author_size),
                title=f"Author: {author}",
                shape="dot",
                color="orange"
            )
            for model in models:
                net.add_edge(author_node, model, color="grey", title="Author Group")

    # 过滤作者为企业的模型（假设你有 `author_type` 属性）
    if view_type == 3:
        filtered_nodes = [
            node for node in visited if eg_graph.nodes[node].get("author_type", "") == "enterprise"
        ]
        net = Network(height="1000px", width="100%", notebook=True, directed=True)
        for node in filtered_nodes:
            add_node_to_net(node, eg_graph.nodes[node], eg_graph)  # 传递 eg_graph 参数
            for successor in eg_graph.successors(node):
                if successor in filtered_nodes:
                    edge_data = eg_graph[node][successor]
                    edge_type = edge_data.get("type", "unknown")
                    edge_color = edge_colors.get(edge_type, "black")
                    net.add_edge(node, successor, color=edge_color, title=edge_type)

    # 过滤按类别展示（假设有 `type` 属性）
    if view_type == 4:
        filtered_nodes = [
            node for node in visited if "adapter" in [eg_graph.nodes[node].get("type", "")]
        ]
        net = Network(height="1000px", width="100%", notebook=True, directed=True)
        for node in filtered_nodes:
            add_node_to_net(node, eg_graph.nodes[node], eg_graph)  # 传递 eg_graph 参数
            for successor in eg_graph.successors(node):
                if successor in filtered_nodes:
                    edge_data = eg_graph[node][successor]
                    edge_type = edge_data.get("type", "unknown")
                    edge_color = edge_colors.get(edge_type, "black")
                    net.add_edge(node, successor, color=edge_color, title=edge_type)

    # 保存为 HTML 文件
    net.show(output_file)

    # 添加视图选择菜单和更新脚本
    menu_html = """
    <div style="position: absolute; top: 10px; left: 10px; z-index: 10; background: white; padding: 10px; border-radius: 5px;">
        <label for="view-selector">Select View:</label>
        <select id="view-selector" onchange="changeView(this.value)">
            <option value="1">Full Network</option>
            <option value="2">Group by Author</option>
            <option value="3">Only Enterprise Models</option>
            <option value="4">Filter by Adapter</option>
        </select>
    </div>
    <script type="text/javascript">
        function changeView(viewType) {
            // Update the view based on the selection
            window.location.href = window.location.pathname + "?view_type=" + viewType;
        }
    </script>
    """
    
    with open(output_file, "r", encoding="utf-8") as f:
        content = f.read()

    # 在文件中插入菜单和侧边栏
    content = content.replace("</body>", menu_html + sidebar_html + "</body>")
    with open(output_file, "w", encoding="utf-8") as f:
        f.write(content)

# 示例：指定一个 base_model 进行可视化
base_model_to_visualize = "meta-llama/Llama-3.1-70B-Instruct"  # 替换为实际 base_model 的名称
visualize_single_base_model(graph, base_model_to_visualize, output_file="single_model_graph.html", view_type=1)


single_model_graph.html


In [89]:
# 动态生成背景颜色及文字颜色
def calculate_color(value, max_value, color_start, color_end, text_color_light="#FFFFFF", text_color_dark="#333333"):
    """
    根据值计算背景颜色和文字颜色，确保对比度。
    value: 当前值
    max_value: 最大值，用于归一化
    color_start: 起始颜色 (RGB)
    color_end: 结束颜色 (RGB)
    text_color_light: 背景深色时使用的浅文字颜色
    text_color_dark: 背景浅色时使用的深文字颜色
    """
    # 非线性映射（sqrt）
    normalized_value = min(1.0, max(0.0, (value / max_value) ** 0.5))  # sqrt(x) 映射到 0~1
    r = int(color_start[0] + (color_end[0] - color_start[0]) * normalized_value)
    g = int(color_start[1] + (color_end[1] - color_start[1]) * normalized_value)
    b = int(color_start[2] + (color_end[2] - color_start[2]) * normalized_value)

    # 计算颜色亮度（Perceived Brightness）
    brightness = (r * 0.299 + g * 0.587 + b * 0.114) / 255
    text_color = text_color_light if brightness < 0.5 else text_color_dark  # 根据亮度选择文字颜色

    # 返回背景颜色和文字颜色
    return f"rgba({r}, {g}, {b}, 0.8)", text_color  # 背景颜色使用透明度 0.8

In [96]:
from pyvis.network import Network
import math
import json

def visualize_single_base_model(eg_graph, base_model, output_file="single_model_graph.html", view_type=1):
    """
    可视化指定 base_model 的子图，支持不同的视图类型。
    Args:
        eg_graph: EasyGraph 的 DiGraph 对象。
        base_model: 要可视化的根节点（base_model）。
        output_file: 输出 HTML 文件的路径。
        view_type: 可视化模式，1：完整网络，2：同作者折叠，3：过滤企业，4：按类别展示
    """
    # 初始化 PyVis 图
    net = Network(height="1000px", width="100%", notebook=True, directed=True)

    # 边的颜色对应衍生类型
    edge_colors = {
        "adapter": "blue",
        "finetune": "green",
        "merge": "red",
        "quantized": "purple",
    }

    # 添加右侧侧边栏（显示节点详细信息）
    sidebar_html = """
    <div id="details" style="position: absolute; right: 10px; top: 10px; width: 450px; height: 700px; overflow-y: auto; background: #ffffff; border: 1px solid #ddd; border-radius: 10px; padding: 20px; box-shadow: 0 2px 6px rgba(0, 0, 0, 0.2);">
        <h4 style="font-family: 'Roboto', sans-serif; font-size: 20px; color: #333; border-bottom: 1px solid #ddd; padding-bottom: 10px; margin-bottom: 20px;">Model Details</h4>
        <div id="node-details" style="font-family: 'Roboto', sans-serif; color: #333; font-size: 14px;"><p>Welcome to LLM Network!</p><p>Click a node to see details.</p></div>
    </div>
    <script type="text/javascript">
        // 监听节点点击事件
        network.on("selectNode", function (params) {
            var nodeId = params.nodes[0];  // 获取节点 ID
            var nodeDetails = nodes.get(nodeId);  // 获取节点数据
            var customDetails = nodeDetails.things_to_show_on_sidebar;  // 获取自定义信息（HTML）
            document.getElementById("node-details").innerHTML = customDetails;  // 更新侧边栏内容
        });
    </script>
    """


    # 定义一个辅助函数，用于添加节点到图中
    def add_node_to_net(node, node_attrs, eg_graph):
        influence = node_attrs.get("influence", 1.0)
        pic = node_attrs.get("pic", "https://huggingface.co/front/assets/huggingface_logo-noborder.svg")
        downloads = node_attrs.get("downloads", 0)
        likes = node_attrs.get("likes", 0)
        author = node_attrs.get("author", "unknown")
        author_full_name = node_attrs.get("author_full_name", author)
        author_type = node_attrs.get("author_type", "unknown")
        
        # 具体化author_type
        if author_type == "usr":
            author_type_on_board = "an individual user"
        else:
            author_type_on_board = "an organization"

        # 判断模型的category
        # 查找指向当前节点的前驱节点（即入边）
        predecessors = list(eg_graph.predecessors(node))  # 获取所有指向该节点的前驱节点 
        if predecessors:
            # 如果有前驱节点，使用前驱节点到当前节点的边的type属性作为category
            edge_type = eg_graph[predecessors[0]][node].get("type", "unknown")  # 获取第一条边的type作为示例
            category = edge_type  # 比如： "adapter of base-model" 或者 "finetune of model-x"
            predecessor = predecessors[0] 
        else:
            # 如果没有前驱节点，默认为 base model
            category = "base model"
        
        # 构建 HuggingFace 模型的网址
        model_url = f"https://huggingface.co/{node}"
        author_url = f"https://huggingface.co/{author}"
        predecessor_url = f"https://huggingface.co/{predecessor}"

        
        # Likes 动态背景颜色
        likes_bg_color, likes_text_color = calculate_color(likes, 1000, (251, 233, 231), (217, 83, 79))

        # Downloads 动态背景颜色
        downloads_bg_color, downloads_text_color = calculate_color(downloads, 300000, (233, 247, 239), (40, 167, 69))

        # Influence 动态背景颜色
        influence_bg_color, influence_text_color = calculate_color(influence, 1000, (255, 249, 231), (240, 173, 78))

        # 更新 HTML 中的背景和文字颜色
        node_info_html = f"""
        <div style="font-family: 'Roboto', sans-serif; color: #333; line-height: 1.6; max-width: 400px; margin: 0 auto;">
            <!-- 顶部图片和标题 -->
            <div style="display: flex; align-items: center; margin-bottom: 20px;">
                <div style="max-width: 70px; max-height: 70px; border-radius: 50%; background: linear-gradient(135deg, #a8edea, #fed6e3); overflow: hidden; display: inline-block; box-shadow: 0 2px 6px rgba(0, 0, 0, 0.2);">
                    <img src="{pic}" alt="{node}" style="width: 100%; height: auto; display: block;">
                </div>

                <h2 style="margin-left: 15px; font-size: 18px; font-weight: bold;">{node}</h2>
            </div>

            <!-- 作者信息 -->
            <div style="background: #f0f0f0; padding: 10px 15px; border-radius: 5px; margin-bottom: 15px; box-shadow: 0 1px 4px rgba(0, 0, 0, 0.1);">
                <p style="margin: 0; font-size: 14px;"><strong>Author:</strong> <a href="{author_url}" target="_blank" style="color: #007bff; text-decoration: none;">{author_full_name}</a> <span style="color: #666;">({author_type_on_board})</span></p>
            </div>

            <!-- 模型类别 -->
            <div style="background: #f0f0f0; padding: 10px 15px; border-radius: 5px; margin-bottom: 15px; box-shadow: 0 1px 4px rgba(0, 0, 0, 0.1);">
                <p style="margin: 0; font-size: 14px;"><strong>Category:</strong> A <span style="font-weight: bold;">{category}</span> version of <a href="{predecessor_url}" target="_blank" style="color: #007bff; text-decoration: none;">{predecessor}</a></p>
            </div>

            <!-- 数据统计信息 -->
            <div style="display: flex; justify-content: space-between; margin-bottom: 15px;">
                <div style="flex: 1; text-align: center; padding: 10px; background: {downloads_bg_color}; color: {downloads_text_color}; border-radius: 5px; margin-right: 10px; box-shadow: 0 1px 4px rgba(0, 0, 0, 0.1);">
                    <p style="margin: 0; font-size: 14px;"><strong>Downloads</strong></p>
                    <p style="margin: 0; font-size: 16px; font-weight: bold;">{downloads}</p>
                </div>
                <div style="flex: 1; text-align: center; padding: 10px; background: {likes_bg_color}; color: {likes_text_color}; border-radius: 5px; margin-left: 10px; box-shadow: 0 1px 4px rgba(0, 0, 0, 0.1);">
                    <p style="margin: 0; font-size: 14px;"><strong>Likes</strong></p>
                    <p style="margin: 0; font-size: 16px; font-weight: bold;">{likes}</p>
                </div>
            </div>

            <!-- 模型影响力 -->
            <div style="background: {influence_bg_color}; color: {influence_text_color}; padding: 15px; border-radius: 5px; margin-bottom: 15px; box-shadow: 0 1px 4px rgba(0, 0, 0, 0.1); text-align: center;">
                <p style="margin: 0; font-size: 14px;"><strong>Influence</strong></p>
                <p style="margin: 0; font-size: 20px; font-weight: bold;">{influence:.2f}</p>
            </div>

            <!-- 模型 URL -->
            <div style="background: #e3f2fd; padding: 10px 15px; border-radius: 5px; margin-bottom: 15px; box-shadow: 0 1px 4px rgba(0, 0, 0, 0.1);">
                <p style="margin: 0; font-size: 14px;"><strong>Model URL:</strong> <a href="{model_url}" target="_blank" style="color: #007bff; text-decoration: none;">{model_url}</a></p>
            </div>
        </div>
        """


        net.add_node(
            node,
            label=node,
            size=math.sqrt(influence),
            title=f"{node} \nby {author}",  # 添加HTML格式的节点信息
            shape="image",
            image=pic,
            things_to_show_on_sidebar=node_info_html  # 添加到侧边栏的数据html代码
        )


    # 深度优先遍历，从 base_model 开始构建子图
    visited = set()  # 用于避免重复访问节点
    stack = [base_model]  # 使用栈模拟深度优先遍历
    author_models = {}  # 存储每个作者的模型，用于折叠

    while stack:
        node = stack.pop()
        if node in visited:
            continue
        visited.add(node)

        node_attrs = eg_graph.nodes[node]
        author = node_attrs.get("author", "unknown")
        
        # 根据选择的视图类型来决定如何处理模型节点
        if view_type == 2:  # 折叠同一个作者的模型
            if author not in author_models:
                author_models[author] = []
            author_models[author].append(node)
            continue  # 不添加节点，等到折叠处理

        # 添加节点到 PyVis 图
        if node not in net.get_nodes():
            add_node_to_net(node, node_attrs, eg_graph)  # 传递 eg_graph 参数

        # 遍历所有子节点（出边）
        for successor in eg_graph.successors(node):
            if successor not in visited and successor not in stack:
                stack.append(successor)

            # 确保目标节点添加到图中
            if successor not in net.get_nodes():
                successor_attrs = eg_graph.nodes[successor]
                add_node_to_net(successor, successor_attrs, eg_graph)  # 传递 eg_graph 参数

            # 添加边
            edge_data = eg_graph[node][successor]
            edge_type = edge_data.get("type", "unknown")
            edge_color = edge_colors.get(edge_type, "black")
            net.add_edge(node, successor, color=edge_color, title=edge_type)

    # 处理视图类型 2: 同一作者的模型折叠
    if view_type == 2:
        for author, models in author_models.items():
            author_size = sum(eg_graph.nodes[m].get("influence", 1.0) for m in models) / len(models)
            author_node = f"{author}_group"
            net.add_node(
                author_node,
                label=author,
                size=math.sqrt(author_size),
                title=f"Author: {author}",
                shape="dot",
                color="orange"
            )
            for model in models:
                net.add_edge(author_node, model, color="grey", title="Author Group")

    # 过滤作者为企业的模型（假设你有 `author_type` 属性）
    if view_type == 3:
        filtered_nodes = [
            node for node in visited if eg_graph.nodes[node].get("author_type", "") == "enterprise"
        ]
        net = Network(height="1000px", width="100%", notebook=True, directed=True)
        for node in filtered_nodes:
            add_node_to_net(node, eg_graph.nodes[node], eg_graph)  # 传递 eg_graph 参数
            for successor in eg_graph.successors(node):
                if successor in filtered_nodes:
                    edge_data = eg_graph[node][successor]
                    edge_type = edge_data.get("type", "unknown")
                    edge_color = edge_colors.get(edge_type, "black")
                    net.add_edge(node, successor, color=edge_color, title=edge_type)

    # 过滤按类别展示（假设有 `type` 属性）
    if view_type == 4:
        filtered_nodes = [
            node for node in visited if "adapter" in [eg_graph.nodes[node].get("type", "")]
        ]
        net = Network(height="1000px", width="100%", notebook=True, directed=True)
        for node in filtered_nodes:
            add_node_to_net(node, eg_graph.nodes[node], eg_graph)  # 传递 eg_graph 参数
            for successor in eg_graph.successors(node):
                if successor in filtered_nodes:
                    edge_data = eg_graph[node][successor]
                    edge_type = edge_data.get("type", "unknown")
                    edge_color = edge_colors.get(edge_type, "black")
                    net.add_edge(node, successor, color=edge_color, title=edge_type)

    # 保存为 HTML 文件
    net.show(output_file)

    # 添加视图选择菜单和更新脚本
    menu_html = """
    <div style="position: absolute; top: 10px; left: 10px; z-index: 10; background: white; padding: 10px; border-radius: 5px;">
        <label for="view-selector">Select View:</label>
        <select id="view-selector" onchange="changeView(this.value)">
            <option value="1">Full Network</option>
            <option value="2">Group by Author</option>
            <option value="3">Only Enterprise Models</option>
            <option value="4">Filter by Adapter</option>
        </select>
    </div>
    <script type="text/javascript">
        function changeView(viewType) {
            // Update the view based on the selection
            window.location.href = window.location.pathname + "?view_type=" + viewType;
        }
    </script>
    """
    
    with open(output_file, "r", encoding="utf-8") as f:
        content = f.read()

    # 在文件中插入菜单和侧边栏
    content = content.replace("</body>", menu_html + sidebar_html + "</body>")
    with open(output_file, "w", encoding="utf-8") as f:
        f.write(content)

# 示例：指定一个 base_model 进行可视化
base_model_to_visualize = "meta-llama/Llama-3.1-70B-Instruct"  # 替换为实际 base_model 的名称
visualize_single_base_model(graph, base_model_to_visualize, output_file="single_model_graph.html", view_type=1)


single_model_graph.html
